In [1]:
#ライブラリの取り込み
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
#DatasetのFormatの作成
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [3]:
#学習データとテストデータの作成
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 25, 25])

In [4]:
#Dataloaderの作成
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [5]:
#Modelの定義
model = models.alexnet(pretrained=True)

In [6]:
#3分類に調整
#種類を増やす場合もここ
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [7]:
#GPUを使う
device = torch.device('cuda')
model = model.to(device)

In [8]:
#学習の開始
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

0: 1.000000
1: 1.000000
2: 1.000000
3: 1.000000
4: 1.000000
5: 1.000000
6: 1.000000
7: 1.000000
8: 1.000000
9: 1.000000
10: 1.000000
11: 1.000000
12: 1.000000
13: 1.000000
14: 1.000000
15: 1.000000
16: 1.000000
17: 1.000000
18: 1.000000
19: 1.000000
20: 1.000000
21: 1.000000
22: 1.000000
23: 1.000000
24: 1.000000
25: 1.000000
26: 1.000000
27: 1.000000
28: 1.000000
29: 1.000000
